In [2]:
import os
import copy
import json
import ast
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import keras
from wordcloud import WordCloud, STOPWORDS
import nltk
import math
import string
from nltk.corpus import stopwords
from collections import Counter
from nltk.stem.porter import *
from sklearn.feature_extraction.text import TfidfVectorizer

%matplotlib inline

Using TensorFlow backend.


In [3]:
# before use data
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [4]:
# get the work directory
os.chdir('/content/gdrive/My Drive/stat628module 3')
# show the files in the wd
print('The work directory contains:')
!ls

The work directory contains:
EDA.ipynb	 final_data.csv  Mexican.ipynb
embedding.ipynb  mexican.csv	 tfidf.csv


In [21]:
mexican = pd.read_csv("mexican.csv")
mexican

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (35,36,38,39,40,42,45,47,48,50,51,52,53,54) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,business_id,name,address,city,state,postal_code,latitude,longitude,stars_x,review_count,is_open,attributes,categories,hours,GoodForKids,RestaurantsReservations,GoodForMeal,BusinessParking,Caters,NoiseLevel,RestaurantsTableService,RestaurantsTakeOut,RestaurantsPriceRange2,OutdoorSeating,BikeParking,Ambience,HasTV,WiFi,Alcohol,RestaurantsAttire,RestaurantsGoodForGroups,RestaurantsDelivery,BusinessAcceptsCreditCards,NoInput,BusinessAcceptsBitcoin,ByAppointmentOnly,AcceptsInsurance,Music,GoodForDancing,CoatCheck,HappyHour,BestNights,WheelchairAccessible,DogsAllowed,BYOBCorkage,DriveThru,Smoking,AgesAllowed,HairSpecializesIn,Corkage,BYOB,DietaryRestrictions,Open24Hours,RestaurantsCounterService,review_id,user_id,stars_y,useful,funny,cool,text,date,year,month
0,0,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True,False,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,True,1.0,False,NaN,"{'romantic': False, 'intimate': False, 'touris...",False,u'no',u'none',u'casual',True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,QNmY376TXxnanuVVOd6jTQ,Hw8mxfICqErjwIVsC5rXUw,3.0,0,0,0,"I mean it's Taco Bell, so it is what it is. Ho...",2018-09-26 03:26:43,2018,9
1,1,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True,False,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,True,1.0,False,NaN,"{'romantic': False, 'intimate': False, 'touris...",False,u'no',u'none',u'casual',True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,q0uAJsHS8tU9Lxjp8HkH6w,qfA3pCsmjRdKl3CYVZ4p7w,1.0,0,1,0,"1 star for service, but the food is not ok :( ...",2012-03-06 14:06:35,2012,3
2,2,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True,False,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,True,1.0,False,NaN,"{'romantic': False, 'intimate': False, 'touris...",False,u'no',u'none',u'casual',True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,s-1UrmPhWuMtTQk-bearcg,ZQsculZs_WMVFnBGQee1EQ,4.0,1,0,0,Taco Bell has really improved over the years a...,2013-03-22 06:17:38,2013,3
3,3,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True,False,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,True,1.0,False,NaN,"{'romantic': False, 'intimate': False, 'touris...",False,u'no',u'none',u'casual',True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,LlZoEaNMmm5V6p_isMr_4g,8YHYABG7zc1s42OAMHAa6Q,5.0,1,1,1,This is one of the most exquisite Taco Belles ...,2017-08-11 03:00:35,2017,8
4,4,1Dfx3zM-rW4n-31KeC8sJg,Taco Bell,2450 E Indian School Rd,Phoenix,AZ,85016,33.495194,-112.028588,3.0,18,1,"{'RestaurantsTakeOut': 'True', 'BusinessParkin...","Restaurants, Breakfast & Brunch, Mexican, Taco...","{'Monday': '7:0-0:0', 'Tuesday': '7:0-0:0', 'W...",True,False,NaN,"{'garage': False, 'street': False, 'validated'...",NaN,NaN,NaN,True,1.0,False,NaN,"{'romantic': False, 'intimate': False, 'touris...",False,u'no',u'none',u'casual',True,False,True,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN

In [0]:
rate = copy.deepcopy(mexican[["business_id","stars_y","text"]])
rate["pn"] = np.sign(rate.stars_y-2.9)
def ratepn(df,pn_value):
  ratepn = df["text"][rate["pn"]==pn_value]
  ratepn = "".join(str(ratepn))
  return ratepn
rate_n = ratepn(rate,-1)
rate_p = ratepn(rate,1)

In [0]:
def get_tokens(text):
    lowers = text.lower()
    #remove the punctuation using the character deletion step of translate
    remove_punctuation_map = dict((ord(char), None) for char in string.punctuation)
    no_punctuation = lowers.translate(remove_punctuation_map)
    tokens = nltk.word_tokenize(no_punctuation)
    return tokens

In [0]:
def stem_tokens(tokens, stemmer):
    stemmed = []
    for item in tokens:
        stemmed.append(stemmer.stem(item))
    return stemmed

In [0]:
def filter(text):
  tokens = get_tokens(text)
  filtered = [w for w in tokens if not w in stopwords.words('english')]
  stemmer = PorterStemmer()
  stemmed = stem_tokens(filtered, stemmer)
  return " ".join(stemmed)

In [26]:
nltk.download('punkt')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [0]:
business = mexican["business_id"].unique()
words_p = list()
words_n = list()
for business_id in business:
  business1 = rate.loc[mexican['business_id']==business_id]
  rate_n = ratepn(business1,-1)
  rate_p = ratepn(business1,1)
  vectorizer = TfidfVectorizer(min_df=1)
  corpus = [rate_p,rate_n]
  vectorizer.fit_transform(corpus)
  words = vectorizer.get_feature_names()
  score_p = vectorizer.fit_transform(corpus).toarray()[0]
  score_n = vectorizer.fit_transform(corpus).toarray()[1]
  df = {'words':words, 'positive':score_p, 'negative':score_n}
  df = pd.DataFrame(df)
  words_p.append(df.sort_values(by=['positive'],ascending=False)[0:100]['words'])
  words_n.append(df.sort_values(by=['negative'],ascending=False)[0:100]["words"])
tfidf = {"business_id":business,'positive words':words_p,'negative words':words_n} 


In [0]:
tfidf = pd.DataFrame(tfidf)
tfidf.to_csv("tfidf.csv")

In [0]:
word_list = ['taco','burrito','salsa','chips','burrito','rice','guacamole','asada','chipotle','tortilla','salad','nacho','enchilada','meat',\
             'chicken','fish','shrimp','beef','pork','steak','service','time','wait','minute','staff','waitress','waiter','atmosphere',\
             'price','internet','wifi','noisy','noise','breakfast','lunch','dinner','line','drink','margarita','table','fries','manager','beer',\
             'patio','bacon','vegan','quesadilla','fajita','chile','enchilada']

In [0]:
pos_words = []
neg_words = []
for i in range(tfidf.shape[0]):
  pos = tfidf["positive words"][i]
  neg = tfidf["negative words"][i]
  pos = pos[pos.isin(word_list)]
  neg = neg[neg.isin(word_list)]
  pos = ','.join(list(set(pos)-set(neg)))
  neg = ','.join(list(set(neg)-set(pos)))
  pos_words.append(pos)
  neg_words.append(neg)
tfidf_words = {'business_id':tfidf['business_id'],'positive words':pos_words, 'negative words':neg_words}
tfidf_words = pd.DataFrame(tfidf_words)
tfidf_words.to_csv("tfidf_words.csv")
#[pos1.isin(word_list)]